In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
import glob

Загрузка

In [3]:
import pandas as pd
import glob

data_frames = []
json_files = glob.glob('Данные по убытию 2/*.json')

for file in json_files:
    try:
        data_frame = pd.read_json(file)
        data_frames.append(data_frame)
    except ValueError as e:
        print(f"Error reading {file}: {e}")


if data_frames:
    df = pd.concat(data_frames, ignore_index=True)
else:
    df = pd.DataFrame()

print(f"Successfully read {len(data_frames)} files.")
print(f"Combined DataFrame shape: {df.shape}")

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000023D6B3973E0>>
Traceback (most recent call last):
  File "C:\Users\MPiskaev\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
                                                 ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Program Files\Python312\Lib\threading.py", line 1533, in enumerate
    def enumerate():
    
KeyboardInterrupt: 


Error reading Данные по убытию 2\departed2023_4.json: Unexpected character found when decoding array value (2)
Successfully read 9 files.
Combined DataFrame shape: (1137839, 48)


Дубликаты и лишние столбцы

In [4]:
df.drop_duplicates()
df.drop(['cnt_no', 'teu', 'contract_no', 'receive_datetime', 'departure_datetime'], axis=1, inplace=True)

Выбросы

In [5]:
Q1 = df['storage_days'].quantile(0.25)
Q3 = df['storage_days'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df[(df['storage_days'] >= lower_bound) & (df['storage_days'] <= upper_bound)]

Обработка дат

In [6]:
date_columns = ['receive_date', 'departure_date', 'departure_date_20_20', 'warehouse_procedure_closed', 'contract_expired']

for i in date_columns:
    df[i] = pd.to_datetime(df[i], errors='coerce')
    # Выделяем месяц, день, год
    df[f'month {i}'] = df[i].dt.month.astype('object')
    df[f'day {i}'] = df[i].dt.day.astype('object')
    df[f'year {i}'] = df[i].dt.year.astype('object')
    df.drop(i, axis=1, inplace=True)

C:\Users\MPiskaev\AppData\Local\Temp\3\ipykernel_6256\2197319818.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[i] = pd.to_datetime(df[i], errors='coerce')


Пропуски

In [7]:
df.fillna('unknown2', inplace=True)

C:\Users\MPiskaev\AppData\Local\Temp\3\ipykernel_6256\4220811746.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unknown2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('unknown2', inplace=True)


Декомпозируем признаки

In [8]:
def split_shipcall(shipcall):
    if shipcall == 'нет данных':
        return pd.Series([None, None, None])
    else:
        parts = shipcall.rsplit(' ', 1)
        vessel = parts[0]
        voyages = parts[1].split('/')
        return pd.Series([vessel, voyages[0], voyages[1]])

df[['shipcall_receive_vessel', 'shipcall_receive_voyage1', 'shipcall_receive_voyage2']] = df['shipcall_receive'].apply(split_shipcall)

df[['shipcall_departure_vessel', 'shipcall_departure_voyage1', 'shipcall_departure_voyage2']] = df['shipcall_departure'].apply(split_shipcall)

df.drop(columns=['shipcall_receive', 'shipcall_departure'], inplace=True)

In [9]:

categorical_columns = df.drop('storage_days', axis=1).columns

df[categorical_columns] = df[categorical_columns].astype('category')

df[categorical_columns] = df[categorical_columns].apply(lambda x: x.cat.codes)

Важность

In [10]:
X = df.drop(columns=['storage_days'])
y = df['storage_days']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

feature_importances = rf_model.feature_importances_
feature_names = X_train.columns

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

threshold = 0.01
important_features = importance_df[importance_df['Importance'] > threshold]['Feature'].tolist()

important_df = df[important_features + ['storage_days']]

print("Количество отобранных признаков:", len(important_features))
print("Новый DataFrame с важными признаками:")
print(important_df.head())

Количество отобранных признаков: 19
Новый DataFrame с важными признаками:
   day receive_date  day departure_date  move_to_direction  is_laden  \
0                22                   4                  3         1   
1                29                   5                  3         1   
2                29                   5                  3         0   
3                23                   5                  3         1   
4                18                   5                  3         0   

   day departure_date_20_20  month receive_date  year receive_date  consignor  \
0                         4                   2                  1        983   
1                         5                   2                  1        572   
2                         5                   2                  1        922   
3                         5                   2                  1        983   
4                         5                   2                  1        560   

   shi

Поиск гиперпараметров

In [12]:
X = important_df.drop(columns=['storage_days'])
y = important_df['storage_days']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
param_grid = {
    'n_estimators': [300, 350],
    'max_depth': [None, 3, 5],
    'min_samples_split': [5,  9],
    'min_samples_leaf': [2, 5],
    'bootstrap': [True, False]
}

rf_model = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=2, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_rf_model = grid_search.best_estimator_
print("Best parameters for RandomForest:", grid_search.best_params_)

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred).astype(int)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mae, mape

rmse, mae, mape = evaluate_model(best_rf_model, X_test, y_test)
print(f"Random Forest: RMSE={rmse}, MAE={mae}, MAPE={mape}")

In [ ]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'n_estimators': [300, 400],
    'max_depth': [None],
    'min_samples_split': [2, 4, 5],
    'min_samples_leaf': [1, 2, 3],
    'bootstrap': [True, False]
}


# Функция для оценки модели
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred).astype(int)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mae, mape

# Функция для тестирования моделей с логированием
def test_rf_model(param_grid, X_train, y_train, X_test, y_test):
    results = []
    for params in ParameterGrid(param_grid):
        print(f"Testing parameters: {params}")
        rf_model = RandomForestRegressor(**params, random_state=42)
        rf_model.fit(X_train, y_train)
        rmse, mae, mape = evaluate_model(rf_model, X_test, y_test)
        print(f"Results - RMSE: {rmse}, MAE: {mae}, MAPE: {mape}")
        results.append((params, rmse, mae, mape))
    return results

# Тестирование моделей
results = test_rf_model(param_grid, X_train, y_train, X_test, y_test)

# Вывод результатов
for params, rmse, mae, mape in results:
    print(f"Parameters: {params}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape}")

# Выбор наилучших параметров
best_params = min(results, key=lambda x: x[1])[0]
print("Best parameters:", best_params)

# Обучение и оценка лучшей модели
best_rf_model = RandomForestRegressor(**best_params, random_state=42)
best_rf_model.fit(X_train, y_train)
rmse, mae, mape = evaluate_model(best_rf_model, X_test, y_test)
print(f"Best Random Forest: RMSE={rmse}, MAE={mae}, MAPE={mape}")

In [15]:
import joblib  

best_params = {
    'n_estimators': 300,
    'max_depth': None,
    'min_samples_split': 5
}

best_rf_model = RandomForestRegressor(**best_params, random_state=42)
best_rf_model.fit(X_train, y_train)





RandomForestRegressor(min_samples_split=5, n_estimators=300, random_state=42)

In [ ]:
# Функция для оценки модели
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred).astype(int)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    
    non_zero_indices = y_test != 0
    y_test_non_zero = y_test[non_zero_indices]
    y_pred_non_zero = y_pred[non_zero_indices]
    mape = np.mean(np.abs((y_test_non_zero - y_pred_non_zero) / y_test_non_zero)) * 100

    return rmse, mae, mape

rmse, mae, mape = evaluate_model(best_rf_model, X_test, y_test)
print(f"Best Random Forest: RMSE={rmse}, MAE={mae}, MAPE={mape}")

In [17]:
# Сохранение модели (опционально)
joblib.dump(best_rf_model, 'best_rf_model.pkl')

['best_rf_model.pkl']